In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr
from tensorflow.keras.models import load_model
from joblib import load
from sklearn.metrics import auc
import joblib
import shutil
from collections import Counter
import random

In [2]:
model = tf.keras.models.load_model('h5/NUEVO2.h5')

# Generar gráficas con valores nuevos

Creación listas

In [3]:
# Lista radio: números del 10 al 52
radio = list(range(5, 50))

# Lista fracvol: tantos unos como números en la lista radio
fracvol = [1 for _ in range(len(radio))]

# Lista espesor: repite valores entre 100 y 120 hasta alcanzar la longitud de la lista radio
espesor = [random.randint(100, 200) for _ in range(len(radio))]


# Ahora las listas radio, fracvol, y espesor están listas para ser usadas
longitud_de_onda = list(range(301, 901))

Checando existencia de la carpeta

In [4]:
# Ruta del directorio
dir_path = 'graficas_nuevas'

# Si el directorio ya existe, eliminarlo
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)

# Crear el directorio
os.makedirs(dir_path)

Este código genera por cada valor de radio, con todos los valores de espesor. También genera un archivo txt que dice que combinación es la que tiene mayor area y menor area. En el título de cada archivo se encuentra el AUC que hay entre 400 y 700 nm, la combinación de parámetros con las que fueron hechas y son clasificadas en 3, baja, mediana y mucha absorción. También incluye el valor máximo entre 400 y 700 nm

In [5]:
# Asegurar que la carpeta existe o crearla
carpeta = dir_path
if not os.path.exists(carpeta):
    os.mkdir(carpeta)

# Asumiendo que todos los arrays (fracvol, radio, espesor) tienen la misma longitud:
num_elementos = len(fracvol)

scaler_X = load('minmax_scaler_X.pkl')

# Lista para almacenar las AUC y sus respectivas combinaciones
auc_info = []

for i in range(num_elementos):
    for j in range(num_elementos):
        data = {
            "Longitud de onda": longitud_de_onda,
            "Fracvol": [fracvol[i]] * len(longitud_de_onda),
            "Radio (nm)": [radio[i]] * len(longitud_de_onda),
            "Espesor de la película (nm)": [espesor[j]] * len(longitud_de_onda)
        }
        
        df = pd.DataFrame(data)
        x = df.values
        x_norm = scaler_X.transform(x)
        predictions = model.predict(x_norm)
        
        df["Índice de absorción"] = predictions
        
        # Calcular AUC para el rango de 400 a 700 nm
        df_filtrado = df[(df['Longitud de onda'] >= 400) & (df['Longitud de onda'] <= 700)]
        valor_auc = auc(df_filtrado['Longitud de onda'], df_filtrado['Índice de absorción'])
        auc_info.append((valor_auc, fracvol[i], radio[i], espesor[j]))

# Crear DataFrame con la información de AUC
df_auc = pd.DataFrame(auc_info, columns=['AUC', 'Fracvol', 'Radio', 'Espesor'])

# Definir las categorías de absorción
bins = [df_auc['AUC'].min(), df_auc['AUC'].quantile(0.33), df_auc['AUC'].quantile(0.66), df_auc['AUC'].max()]
labels = ['Poca absorción', 'Mediana absorción', 'Mucha absorción']
categorias = pd.cut(df_auc['AUC'], bins=bins, labels=labels)

for idx, row in df_auc.iterrows():
    valor_auc, fracvol_i, radio_i, espesor_j = row
    
    data = {
        "Longitud de onda": longitud_de_onda,
        "Fracvol": [fracvol_i] * len(longitud_de_onda),
        "Radio (nm)": [radio_i] * len(longitud_de_onda),
        "Espesor de la película (nm)": [espesor_j] * len(longitud_de_onda)
    }
    
    df = pd.DataFrame(data)
    x = df.values
    x_norm = scaler_X.transform(x)
    predictions = model.predict(x_norm)
    
    df["Índice de absorción"] = predictions
    
    # Filtrar datos entre 400 a 700 nm para encontrar el valor máximo
    df_filtrado = df[(df['Longitud de onda'] >= 400) & (df['Longitud de onda'] <= 700)]
    
    # Crear la gráfica de dispersión
    plt.figure(figsize=(10, 6))
    ax = plt.gca()
    ax.plot(df['Longitud de onda'], df['Índice de absorción'], color='blue')
    ax.set_xlabel('Longitud de onda')
    ax.set_ylabel('Índice de absorción')
    titulo = f'fracción volumen: {fracvol_i}, radio: {radio_i}, espesor: {espesor_j}, AUC: {valor_auc:.5f}, {categorias[idx]}'
    ax.set_title(titulo)
    ax.grid(True)
    
    # Indicar el punto máximo
    max_idx = df_filtrado['Índice de absorción'].idxmax()
    max_x = df_filtrado['Longitud de onda'][max_idx]
    max_y = df_filtrado['Índice de absorción'][max_idx]
    ax.annotate(f'Valor máximo: {max_y:.5f}', 
                xy=(max_x, max_y), 
                xytext=(-10, 10), 
                textcoords='offset points', 
                arrowprops=dict(arrowstyle='->', lw=1.5),
                fontsize=10,
                ha='right')
    
    # Guardar la gráfica como imagen en la carpeta especificada
    nombre_archivo = os.path.join(carpeta, f"grafica_{fracvol_i:.5f}_{radio_i:.5f}_{espesor_j:.5f}.png")
    plt.savefig(nombre_archivo)
    plt.close()


# Escribir información en archivo txt
with open(os.path.join(carpeta, 'info_auc.txt'), 'w') as f:
    f.write(f'Mínimo AUC: {df_auc["AUC"].min():.2f}, Combinación: {df_auc.iloc[df_auc["AUC"].idxmin()][1:].to_list()}\n')
    f.write(f'Máximo AUC: {df_auc["AUC"].max():.2f}, Combinación: {df_auc.iloc[df_auc["AUC"].idxmax()][1:].to_list()}\n')
    f.write(f'Promedio AUC: {df_auc["AUC"].mean():.2f}\n')
    f.write(f'Desviación Estándar AUC: {df_auc["AUC"].std():.2f}\n')


19/19 [==============================] - 0s 510us/step


# Asegurar que la carpeta existe o crearla
carpeta = dir_path
if not os.path.exists(carpeta):
    os.mkdir(carpeta)

# Asumiendo que todos los arrays (fracvol, radio, espesor) tienen la misma longitud:
num_elementos = len(fracvol)

scaler_X = load('minmax_scaler_X.pkl')

for i in range(num_elementos):
    for j in range(num_elementos):
        data = {
            "Longitud de onda": longitud_de_onda,
            "Fracvol": [fracvol[i]] * len(longitud_de_onda),
            "Radio (nm)": [radio[i]] * len(longitud_de_onda),
            "Espesor de la película (nm)": [espesor[j]] * len(longitud_de_onda)
        }
        
        df = pd.DataFrame(data)
        x = df.values
        x_norm = scaler_X.transform(x)
        predictions = model.predict(x_norm)

        df["Índice de absorción"] = predictions

        # Filtrar datos entre 400 a 700 nm
        df_filtrado = df[(df['Longitud de onda'] >= 400) & (df['Longitud de onda'] <= 700)]
        
        # Calcular AUC
        valor_auc = auc(df_filtrado['Longitud de onda'], df_filtrado['Índice de absorción'])

        columnas_seleccionadas = ['Longitud de onda', 'Índice de absorción']
        df_seleccionado = df[columnas_seleccionadas]
        titulo = f'fracción volumen: {fracvol[i]:.2f}, radio: {radio[i]:.2f}, espesor: {espesor[j]:.2f}, AUC: {valor_auc:.2f}'

        # Crear la gráfica de dispersión
        plt.figure(figsize=(10, 6))
        plt.plot(df_seleccionado['Longitud de onda'], df_seleccionado['Índice de absorción'], color='blue')
        plt.xlabel('Longitud de onda')
        plt.ylabel('Índice de absorción')
        plt.title(titulo)
        plt.grid(True)
        
        # Guardar la gráfica como imagen en la carpeta especificada
        nombre_archivo = os.path.join(carpeta, f"grafica_{fracvol[i]}_{radio[i]}_{espesor[j]}.png")
        plt.savefig(nombre_archivo)
        plt.close()